In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import glob
import torch
from torch.utils import data
from PIL import Image
import numpy as np
from torchvision import transforms
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

### details: 
##### - load in data
##### - write our own classification neural network
##### - testing OpenCV
##### - importing pretrained model (VGG16)
##### - Incorporating dataset with VGG16



In [ ]:
info = pd.read_csv("clothing-dataset-master/images.csv")
info


In [ ]:
def convert_data(path):

    image = Image.open(path)
    bw_image = image.convert('L')
    bw_image = bw_image.resize((96, 96))
    pixel_values = np.array(bw_image).flatten()
    pixel_list = pixel_values.tolist()

    return pixel_list

In [ ]:
all_imgs_path = glob.glob(r'/Users/clairejaroonjetjumnong/Documents/Projects/NewHacks/clothing-dataset-master/images/*')
img_names = [img.split('/')[-1] for img in all_imgs_path]
img_names = [img.split('.')[0] for img in img_names]

big_list = [convert_data(path) for path in all_imgs_path]
df = pd.DataFrame(big_list)
df['image'] = img_names
for img in img_names:
    matching_rows = info[info['image'] == img]
    if not matching_rows.empty:
        label = matching_rows['label'].values[0]
        df.loc[df['image'] == img, 'label'] = label
    else:
        # Handle the case where there's no matching entry in 'info' for the image
        print(f"No label found for image: {img}")

df


In [ ]:
df = df.dropna(how='any')
df

In [ ]:
unique_values = df['label'].unique().tolist()
print(unique_values)
print(df['label'].value_counts())
labels_to_exclude = ['Not sure', 'Longsleeve','Hat','Skirt', 'Polo', 'Undershirt', 'Blazer', 'Hoodie', 'Body', 'Other', 'Top', 'Blouse', 'Skip']
df = df[~df['label'].isin(labels_to_exclude)]

In [ ]:
transform = transforms.Compose([
                transforms.Resize((96,96)), 
                transforms.ToTensor(), 
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) #Normalize the image #RGB 3 channels withnin [-1,1]
])

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to the 'label' column
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Fit the label encoder to the class labels and transform them to numerical values
encoded_labels = label_encoder.fit_transform(df['label'])

# Get the mapping between class labels and their corresponding numerical values
label_mapping = {encoded_label: label for label, encoded_label in zip(df['label'], encoded_labels)}
print(label_mapping)


In [ ]:
df.to_csv("96clothes.csv", index=False)

In [ ]:
all_imgs_path = [f'/Users/clairejaroonjetjumnong/Documents/Projects/NewHacks/clothing-dataset-master/images/{img}.jpg' for img in df['image']]
all_labels = df['label_encoded'].to_list()

from torch.utils import data

class DataSetClass(data.Dataset):
    def __init__(self, img_paths, labels, transform):
        self.imgs = img_paths
        self.labels = labels
        self.transforms = transform
    def __getitem__(self, index):               
        img = self.imgs[index]                  #Slice the data according to the index, and then return the data -> to tensor
        label = self.labels[index]
        pil_img = Image.open(img).convert('RGB')             
        data = self.transforms(pil_img)
        return data, label
    def __len__(self):
        return len(self.imgs)

BATCH_SIZE = 10
food_dataset = DataSetClass(all_imgs_path, all_labels, transform)
food_datalodaer = data.DataLoader(
                            food_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle=True
)
# Visualize the data
imgs_batch, labels_batch = next(iter(food_datalodaer))
print(imgs_batch.shape)

plt.figure(figsize=(12, 8))
for i, (img, label) in enumerate(zip(imgs_batch[:6], labels_batch[:6])):
    img = img.permute(1, 2, 0).numpy()
    plt.subplot(2, 3, i+1)
    plt.imshow(img)
    plt.title(f"Class: {all_labels[label]}")  # Display the label as the title

plt.show()

In [ ]:
index = np.random.permutation(len(all_imgs_path))

all_imgs_path = np.array(all_imgs_path)[index]
all_labels = np.array(all_labels)[index]
print("Total Number of Samples: ",len(all_imgs_path))

#80% as train
s = int(len(all_imgs_path)*0.7)
print("Numbers of Pic for the Training Set: ",s)

train_imgs = all_imgs_path[:s]
train_labels = all_labels[:s]
test_imgs = all_imgs_path[s:]
test_labels = all_labels[s:]

trainset = DataSetClass(train_imgs, train_labels, transform) #TrainSet TensorData
testset = DataSetClass(test_imgs, test_labels, transform) #TestSet TensorData
trainloader = data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True) #TrainSet Labels
testloader = data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True) #TestSet Labels

In [ ]:
unique_values = df['label'].unique()
print(len(unique_values))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) #input: 3, output: 6, kernel_size: 5, current img size: 92
        self.pool = nn.MaxPool2d(2, 2) #kernel_size: 2, stride: 2, , current img size: 46
        self.conv2 = nn.Conv2d(6, 16, 5) #input: 6, output: 16, kernel_size: 5, current img size: 42
        self.fc1 = nn.Linear(16 * 21 * 21, 120) #input: 16*21*21, output: 120
        self.fc2 = nn.Linear(120, 84) #input: 120, output: 84
        self.fc3 = nn.Linear(84, len(label_mapping)) #input:84, output:2 -> 2 classes

    def forward(self, x):
        # apply the convolutional layer followed by relu activation and max-pooling
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # flatten the output tensor before passing it to the fully connected layers
        x = x.view(-1, 16 * 21 * 21)
        # apply the fully connected later followed by relu activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # apply the last fully connected layer with no activation function
        x = self.fc3(x)
        return x

    
# create an instance of the net class and send it to the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net().to(device)
print(net)


In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# use cross entropy loss because easy to compare probability distribution, closer to the correct label means lower loss
criterion = nn.CrossEntropyLoss()
# Adam (Adaptive learning and Momentum)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

# loop over the dataset for 20 epochs
for epoch in range(15):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if device.type == 'cuda':
            inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # apply the model
        outputs = net(inputs)
        # calculate loss
        loss = criterion(outputs, labels)
        
        # back propagation
        loss.backward()
        # update the model's parameters
        optimizer.step()

        # print statistics
        running_loss += loss.item()

        # printing epoch, batch, running lsoss
        if i % 20 == 19: 
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 20:.3f}')
            running_loss = 0.0
print('Finished Training')

In [ ]:
# test it on the entire validation set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        if device.type == 'cuda':
            images = images.to(device)
            inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in unique_values}
total_pred = {classname: 0 for classname in unique_values}

preds = []
trues = []
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            preds.append(prediction)
            trues.append(label)
            if label == prediction:
                correct_pred[unique_values[label]] += 1
            total_pred[unique_values[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname} is {accuracy} %')

preds = [i.item() for i in preds]
trues = [i.item() for i in trues]

# T-Shirt       1011
# Pants          692
# Shoes          431
# Shirt          378
# Dress          357
# Outwear        312
# Shorts         308

In [ ]:
# show result
import random
imgs_batch, labels_batch = next(iter(testloader))
index0 = random.randint(0,31)
img,label = imgs_batch[0], labels_batch[0]
img = img.to(device)
outputs = net(img)
_, predicted = torch.max(outputs.data, 1)
print('True class:', label_mapping[int(label)], 'predicted:', label_mapping[int(predicted)])

img = img.cpu()
img = img.permute(1, 2, 0).numpy()
# plt.subplot(2, 3)
plt.imshow(img)
plt.show()

In [ ]:
from torch.utils import data

path = 'archive/hoodietest2.jpeg'

# TestDataset class that resembles the Dataset class above but no labels
class TestDataset(data.Dataset):
    def __init__(self, file_paths, transform=None):
        self.file_paths = file_paths
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, index):
        img_path = self.file_paths[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

test_imgs_path = glob.glob(path)
BATCH_SIZE = 10

transform = transforms.Compose([
                transforms.Resize((96,96)), 
                transforms.ToTensor(), 
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_dataset = TestDataset(test_imgs_path, transform=transform)
testloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

results = []
# evaluation
net.eval()
with torch.no_grad():
    for batch in testloader:
        inputs = batch
        inputs = inputs.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        results.extend(predicted.cpu().numpy())
        

print(label_mapping[int(results[0])])

from PIL import Image
import matplotlib.pyplot as plt

# Open and display the image
image = Image.open(path)
plt.imshow(image)
plt.axis('off')  # Turn off axis labels and ticks
plt.show()


In [ ]:
import torch
model_path = 'fashionlol.pth'
torch.save(net.state_dict(), model_path)

In [ ]:
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
import cv2
from PIL import Image

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

plt.imshow(frame_rgb)
plt.axis('off')  # Turn off axis labels and ticks
plt.show()

In [ ]:
# Get the shape of the image data
height, width, channels = frame_rgb.shape

# Calculate the coordinates of the center pixel
center_x = width // 2
center_y = height // 2

# Get the pixel values at the center
center_pixel = frame_rgb[center_y, center_x]

# 'center_pixel' contains the RGB values of the center pixel
print('Center Pixel RGB:', center_pixel)

color_image = [[center_pixel]]
# Display the color using Matplotlib
plt.imshow(color_image)
plt.axis('off')  # Hide the axes
plt.show()

In [ ]:
print(type(frame))

In [ ]:
def get_class_from_screen_shot(frame):
    '''return classification from '''
    transform = transforms.Compose([
                    transforms.Resize((96,96)), 
                    transforms.ToTensor(), 
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    pil_image = Image.fromarray((frame * 255).astype('uint8'))
    transformed_tensor = transform(pil_image)
    model = Net()
    model.load_state_dict(torch.load('fashionlol.pth'))
    model.eval()
    with torch.no_grad():
        for batch in testloader:
            inputs = batch
            inputs = inputs.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            results.extend(predicted.cpu().numpy())
            
    label_mapping = {encoded_label: label for label, encoded_label in zip(df['label'], encoded_labels)}
    return label_mapping[int(results[0])]

def get_class_from_screen_shot(frame):
    '''return classification from '''
    transform = transforms.Compose([
                    transforms.Resize((96,96)), 
                    transforms.ToTensor(), 
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    pil_image = Image.fromarray((frame * 255).astype('uint8'))
    transformed_tensor = transform(pil_image)
    model = Net()
    model.load_state_dict(torch.load('fashionlol.pth'))
    model.eval()
    with torch.no_grad():
        for batch in testloader:
            inputs = batch
            inputs = inputs.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            results.extend(predicted.cpu().numpy())
            
    label_mapping = {encoded_label: label for label, encoded_label in zip(df['label'], encoded_labels)}
    return label_mapping[int(results[0])]

get_class_from_screen_shot(frame_rgb)

In [ ]:
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import glob
import torch
from torch.utils import data
import numpy as np
from torchvision import transforms
import torchvision
import torch.nn as nn
import torch.nn.functional as F

def path_classi(path):

    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 6, 5) #input: 3, output: 6, kernel_size: 5, current img size: 92
            self.pool = nn.MaxPool2d(2, 2) #kernel_size: 2, stride: 2, , current img size: 46
            self.conv2 = nn.Conv2d(6, 16, 5) #input: 6, output: 16, kernel_size: 5, current img size: 42
            self.fc1 = nn.Linear(16 * 21 * 21, 120) #input: 16*21*21, output: 120
            self.fc2 = nn.Linear(120, 84) #input: 120, output: 84
            self.fc3 = nn.Linear(84, 7) #input:84, output:7 -> 7 classes

        def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 16 * 21 * 21)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x

    class TestDataset(data.Dataset):
        def __init__(self, file_paths, transform=None):
            self.file_paths = file_paths
            self.transform = transform

        def __len__(self):
            return len(self.file_paths)

        def __getitem__(self, index):
            img_path = self.file_paths[index]
            img = Image.open(img_path).convert('RGB')
            if self.transform:
                img = self.transform(img)
            return img
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    test_imgs_path = glob.glob(path)
    BATCH_SIZE = 10

    transform = transforms.Compose([
                    transforms.Resize((96,96)), 
                    transforms.ToTensor(), 
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    test_dataset = TestDataset(test_imgs_path, transform=transform)
    testloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    results = []
    model = Net()
    model.load_state_dict(torch.load('fashionlol.pth'))
    model.eval()
    with torch.no_grad():
        for batch in testloader:
            inputs = batch
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            results.extend(predicted.cpu().numpy())
            
    label_mapping = {1: 'Longsleeve', 7: 'T-Shirt', 6: 'Shorts', 5: 'Shoes', 4: 'Shirt', 0: 'Dress', 3: 'Pants', 2: 'Outwear'}
    return label_mapping[int(results[0])]



In [ ]:
import numpy as np
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from PIL import Image

def path_classi(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))
    x = np.array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    model = VGG16(weights='imagenet')

    predictions = model.predict(x)

    decoded_predictions = decode_predictions(predictions, top=5)[0]
    
    return decoded_predictions[0][1]

path = "archive/myclothes.jpg"
image = Image.open(path)
plt.imshow(image)
plt.axis('off')
plt.show()
path_classi(path)


In [ ]:
# from torch.utils import data
# import numpy as np
# import pandas as pd
# from PIL import Image
# import matplotlib.pyplot as plt
# import glob
# import torch
# from torch.utils import data
# from PIL import Image
# import numpy as np
# from torchvision import transforms
# import torchvision
# import torch.nn as nn
# import torch.nn.functional as F

# def path_classi(path):

#     class Net(nn.Module):
#         def __init__(self):
#             super(Net, self).__init__()
#             self.conv1 = nn.Conv2d(3, 6, 5) #input: 3, output: 6, kernel_size: 5, current img size: 92
#             self.pool = nn.MaxPool2d(2, 2) #kernel_size: 2, stride: 2, , current img size: 46
#             self.conv2 = nn.Conv2d(6, 16, 5) #input: 6, output: 16, kernel_size: 5, current img size: 42
#             self.fc1 = nn.Linear(16 * 21 * 21, 120) #input: 16*21*21, output: 120
#             self.fc2 = nn.Linear(120, 84) #input: 120, output: 84
#             self.fc3 = nn.Linear(84, 7) #input:84, output:2 -> 2 classes

#         def forward(self, x):
#             # apply the convolutional layer followed by relu activation and max-pooling
#             x = self.pool(F.relu(self.conv1(x)))
#             x = self.pool(F.relu(self.conv2(x)))
#             # flatten the output tensor before passing it to the fully connected layers
#             x = x.view(-1, 16 * 21 * 21)
#             # apply the fully connected later followed by relu activation
#             x = F.relu(self.fc1(x))
#             x = F.relu(self.fc2(x))
#             # apply the last fully connected layer with no activation function
#             x = self.fc3(x)
#             return x

#     class TestDataset(data.Dataset):
#         def __init__(self, file_paths, transform=None):
#             self.file_paths = file_paths
#             self.transform = transform

#         def __len__(self):
#             return len(self.file_paths)

#         def __getitem__(self, index):
#             img_path = self.file_paths[index]
#             img = Image.open(img_path).convert('RGB')
#             if self.transform:
#                 img = self.transform(img)
#             return img
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#     test_imgs_path = glob.glob(path)
#     BATCH_SIZE = 10

#     transform = transforms.Compose([
#                     transforms.Resize((96,96)), 
#                     transforms.ToTensor(), 
#                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ])

#     test_dataset = TestDataset(test_imgs_path, transform=transform)
#     testloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

#     results = []
#     model = Net()
#     model.load_state_dict(torch.load('fashionlol.pth'))
#     model.eval()
#     with torch.no_grad():
#         for batch in testloader:
#             inputs = batch
#             inputs = inputs.to(device)
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs, 1)
#             results.extend(predicted.cpu().numpy())
            
#     label_mapping = {1: 'Longsleeve', 7: 'T-Shirt', 6: 'Shorts', 5: 'Shoes', 4: 'Shirt', 0: 'Dress', 3: 'Pants', 2: 'Outwear'}
#     return label_mapping[int(results[0])]



In [138]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import glob

data = pd.read_csv("96clothes.csv")

all_imgs_path = [f'/Users/clairejaroonjetjumnong/Documents/Projects/NewHacks/clothing-dataset-master/images/{img}.jpg' for img in data['image']]
all_labels = data['label_encoded'].to_list()

image_width = 224
image_height = 224
batch_size = 32
num_classes = 7

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(image_width, image_height))
    img = img_to_array(img)
    img = img / 255.0
    return img

labels = to_categorical(all_labels, num_classes=num_classes)

def data_generator(image_paths, labels, batch_size):
    num_samples = len(image_paths)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_paths = image_paths[i : i + batch_size]
            batch_labels = labels[i : i + batch_size]
            
            batch_images = [preprocess_image(image_path) for image_path in batch_paths]
            yield np.array(batch_images), batch_labels

train_generator = data_generator(all_imgs_path, labels, batch_size)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(num_classes, activation='softmax')(x)

# Create the custom model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on your custom dataset
model.fit(
    train_generator,
    steps_per_epoch=len(all_imgs_path) // batch_size,
    epochs=10,
    verbose=1
)

# Save the trained model
model.save('custom_clothing_vgg16.h5')

 61/109 [===============>..............] - ETA: 21:13 - loss: 1.8755 - accuracy: 0.2785